In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [35]:
###############################################logistic with regularization, regularization parameter= 1 ##################################################
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :]) #每次取10000个数据进行训练
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels])) #初始化矩阵w，大小为784*10
  biases = tf.Variable(tf.zeros([num_labels]))                  #初始化偏置变量
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases        #y=w*x+b+ reglarization
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) #logits代表通过矩阵运算出来的数据结果，labels代表实际的标签
                                                                                     #通过softmax计算的结果后通过减小交叉熵来进行运算。           
  loss =loss + 0.01*tf.nn.l2_loss(weights)
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)#通过gradient descent来优化交叉熵。0.5代表的意思不确定，有可能是优化的速率。下降的速率 
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits) #softmax函数算出来的结果应该是一个矩阵
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  print("definition of data is finished")

num_steps = 12001


with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):   #step代表优化的次数。相当于每个step只优化一步。意思是每一步中，对这10000个subset算出loss，然后根据loss改变
    # Run the computations. We tell .run() that we want to run the optimizer,     一次矩阵w中的系数和偏置b的值。自己选择计算的次数多少次优化。
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))            #这里只是代表每次只取train_data的前10000个数据进行prediction。
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

definition of data is finished
Initialized
Loss at step 0: 46.209663
Training accuracy: 9.0%
Validation accuracy: 11.9%
Loss at step 100: 11.496471
Training accuracy: 74.6%
Validation accuracy: 73.0%
Loss at step 200: 4.356350
Training accuracy: 80.0%
Validation accuracy: 77.3%
Loss at step 300: 1.927995
Training accuracy: 82.9%
Validation accuracy: 80.7%
Loss at step 400: 1.100533
Training accuracy: 84.4%
Validation accuracy: 82.0%
Loss at step 500: 0.815360
Training accuracy: 84.8%
Validation accuracy: 82.3%
Loss at step 600: 0.715989
Training accuracy: 85.0%
Validation accuracy: 82.5%
Loss at step 700: 0.681057
Training accuracy: 85.0%
Validation accuracy: 82.7%
Loss at step 800: 0.668684
Training accuracy: 85.1%
Validation accuracy: 82.7%
Loss at step 900: 0.664267
Training accuracy: 85.1%
Validation accuracy: 82.7%
Loss at step 1000: 0.662674
Training accuracy: 85.1%
Validation accuracy: 82.7%
Loss at step 1100: 0.662090
Training accuracy: 85.1%
Validation accuracy: 82.7%
Loss at 

In [12]:
###################################################regularization##################################################################

batch_size = 128
train_subset = 200000

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  
  
  # Training computation.
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  loss = loss + 0.0005*tf.nn.l2_loss(weights1)+ 0.0005*tf.nn.l2_loss(weights2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)


# In[23]:

num_steps = 80001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print ("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels[:train_subset, :].shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[:train_subset, :][offset:(offset + batch_size), :]
    batch_labels = train_labels[:train_subset, :][offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 486.422
Minibatch accuracy: 11.7%
Validation accuracy: 47.1%
Minibatch loss at step 1000 : 95.6608
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 2000 : 57.3957
Minibatch accuracy: 85.9%
Validation accuracy: 83.3%
Minibatch loss at step 3000 : 33.9853
Minibatch accuracy: 83.6%
Validation accuracy: 85.0%
Minibatch loss at step 4000 : 20.7886
Minibatch accuracy: 83.6%
Validation accuracy: 86.6%
Minibatch loss at step 5000 : 12.921
Minibatch accuracy: 85.2%
Validation accuracy: 87.5%
Minibatch loss at step 6000 : 7.67541
Minibatch accuracy: 94.5%
Validation accuracy: 88.5%
Minibatch loss at step 7000 : 4.96014
Minibatch accuracy: 93.8%
Validation accuracy: 88.8%
Minibatch loss at step 8000 : 3.24793
Minibatch accuracy: 85.9%
Validation accuracy: 89.0%
Minibatch loss at step 9000 : 2.08002
Minibatch accuracy: 90.6%
Validation accuracy: 89.2%
Minibatc

In [15]:
###################################################dropout##################################################################

batch_size = 128
train_subset = 3000

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden1  = tf.nn.dropout(hidden1, 0.5)
  
  
  # Training computation.
  logits = tf.matmul(hidden1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)


# In[23]:

num_steps = 50001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print ("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels[:train_subset, :].shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[:train_subset, :][offset:(offset + batch_size), :]
    batch_labels = train_labels[:train_subset, :][offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 453.69
Minibatch accuracy: 7.8%
Validation accuracy: 28.0%
Minibatch loss at step 1000 : 1.17043
Minibatch accuracy: 96.1%
Validation accuracy: 81.7%
Minibatch loss at step 2000 : 7.72764
Minibatch accuracy: 97.7%
Validation accuracy: 80.2%
Minibatch loss at step 3000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 82.4%
Minibatch loss at step 4000 : 0.269685
Minibatch accuracy: 97.7%
Validation accuracy: 82.0%
Minibatch loss at step 5000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 82.0%
Minibatch loss at step 6000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 82.1%
Minibatch loss at step 7000 : 3.28086e-05
Minibatch accuracy: 100.0%
Validation accuracy: 82.2%
Minibatch loss at step 8000 : 3.05786e-06
Minibatch accuracy: 100.0%
Validation accuracy: 82.2%
Minibatch loss at step 9000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 82.2%
Minibatch 

In [51]:
###################################################2 hidden layer##################################################################

batch_size = 128
train_subset = 200000

hidden_layer_1_stddev = np.sqrt(2.0/784) 
hidden_layer_2_stddev = np.sqrt(2.0/1024)
hidden_layer_3_stddev = np.sqrt(2.0/512)
output_layer_stddev = np.sqrt(2.0/256)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024],stddev=hidden_layer_1_stddev))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, 512],stddev=hidden_layer_2_stddev))
  biases2 = tf.Variable(tf.zeros([512]))

  weights3 = tf.Variable(
    tf.truncated_normal([512, 256],stddev=hidden_layer_3_stddev))
  biases3 = tf.Variable(tf.zeros([256]))

  weights4 = tf.Variable(
    tf.truncated_normal([256, num_labels],stddev=output_layer_stddev))
  biases4 = tf.Variable(tf.zeros([num_labels]))

  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1) 
  hidden1 = tf.nn.dropout(hidden1,0.5)

  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
  hidden2 = tf.nn.dropout(hidden2,0.8)

  hidden3 = tf.nn.relu(tf.matmul(hidden2, weights3) + biases3)
  hidden3 = tf.nn.dropout(hidden3,0.9)
  
  
  # Training computation.
  logits = tf.matmul(hidden3, weights4) + biases4
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  #loss = loss + (tf.nn.l2_loss(weights1)+ tf.nn.l2_loss(weights2)+tf.nn.l2_loss(weights3)+tf.nn.l2_loss(weights4))*0.0001

  
  
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 10000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction_3 = tf.nn.relu( 
     tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2),weights3) + biases3)
  valid_prediction  = tf.nn.softmax(tf.matmul(valid_prediction_3,weights4)+biases4)
  test_prediction_3  =  tf.nn.relu(
          tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2),weights3)  + biases3)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction_3,weights4)+biases4)

# In[23]:

num_steps = 80001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print ("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels[:train_subset, :].shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[:train_subset, :][offset:(offset + batch_size), :]
    batch_labels = train_labels[:train_subset, :][offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10000 == 0):
      print ("Minibatch loss at step", step, ":", l)
      print ("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print ("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print ("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 2.4661
Minibatch accuracy: 7.8%
Validation accuracy: 26.5%
Test accuracy: 28.2%
Minibatch loss at step 10000 : 0.359764
Minibatch accuracy: 89.1%
Validation accuracy: 90.3%
Test accuracy: 95.1%
Minibatch loss at step 20000 : 0.273591
Minibatch accuracy: 92.2%
Validation accuracy: 90.8%
Test accuracy: 95.8%
Minibatch loss at step 30000 : 0.274634
Minibatch accuracy: 91.4%
Validation accuracy: 91.3%
Test accuracy: 96.1%
Minibatch loss at step 40000 : 0.165263
Minibatch accuracy: 93.8%
Validation accuracy: 91.4%
Test accuracy: 96.3%
Minibatch loss at step 50000 : 0.106434
Minibatch accuracy: 96.1%
Validation accuracy: 91.8%
Test accuracy: 96.3%
Minibatch loss at step 60000 : 0.142219
Minibatch accuracy: 95.3%
Validation accuracy: 91.8%
Test accuracy: 96.4%
Minibatch loss at step 70000 : 0.188758
Minibatch accuracy: 93.0%
Validation accuracy: 91.9%
Test accuracy: 96.3%
Minibatch